<a href="https://colab.research.google.com/github/BenBass91/tree-of-thought-benchmarking/blob/main/R1_ToT_FineTune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run in terminal:

```
sudo apt-get update -y && sudo apt-get install -y git-lfs
git lfs install
```

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 40.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip -q install "unsloth>=2024.7.5" "peft>=0.10.0" "transformers>=4.43" "trl>=0.9.6" \
                 "accelerate>=0.31" "datasets>=2.20.0" "bitsandbytes>=0.43.1"
import torch, transformers, trl, unsloth, peft, datasets, bitsandbytes
print("unsloth:", unsloth.__version__)
print("transformers:", transformers.__version__)
print("trl:", trl.__version__)


unsloth: 2025.9.1
transformers: 4.57.0.dev0
trl: 0.22.2


In [ ]:
# 1) Imports (use Unsloth’s trainer)
from unsloth import FastLanguageModel
from unsloth.tokenizer_utils import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, TrainingArguments
import torch


In [ ]:
from huggingface_hub import login, whoami
from google.colab import userdata

HF_TOKEN= userdata.get('HF_TOKEN')
login(token=HF_TOKEN)
print(whoami())  # sanity check your username/orgs


{'type': 'user', 'id': '682206638708b390d9f2c709', 'name': 'Benbass1991', 'fullname': 'Benjamin Soldate', 'email': 'bensoldate@gmail.com', 'emailVerified': True, 'canPay': True, 'periodEnd': 1759276799, 'isPro': True, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/no-auth/tY1JVASdz7V4GD3xR36MU.png', 'orgs': [{'type': 'org', 'id': '682206ef3bfd41d412b9a7b9', 'name': 'Sunflower91502', 'fullname': 'Sunflower Labs ', 'email': None, 'canPay': False, 'periodEnd': None, 'avatarUrl': 'https://www.gravatar.com/avatar/58c902ff3015c7d5f08a6d60f7181415?d=retro&size=100', 'roleInOrg': 'admin', 'isEnterprise': False}], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'GoogleColab', 'role': 'write', 'createdAt': '2025-09-04T21:37:45.077Z'}}}


In [ ]:
model_name = "r1_ToT_FineTune"
hub_model_id = f"Benbass1991/{model_name}"

In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.create_repo(repo_id=hub_model_id, repo_type="model", exist_ok=True, token=HF_TOKEN)


RepoUrl('https://huggingface.co/Benbass1991/r1_ToT_FineTune', endpoint='https://huggingface.co', repo_type='model', repo_id='Benbass1991/r1_ToT_FineTune')

In [ ]:
from datasets import load_dataset
# Load the Tree-of-Thoughts dataset (or any HF dataset by name)
dataset_name = "terrycraddock/Tree_Of_Thoughts_BASE_24k"

# Slice for testing (first 500 rows), remove split if you want full dataset
raw_ds = load_dataset(dataset_name, split="train[0:500]", trust_remote_code=True)

# Peek at one example
print(raw_ds[0])

{'Question': '"What was the primary technique employed to prevent overfitting during model training?"', 'Answer': "<thinking>\n1. Break down the question: The main question requires understanding the techniques used to prevent overfitting during model training.\n2. Identify possible sub-questions:\n   - What is overfitting and why does it occur?\n   - What are some common techniques used to prevent overfitting?\n\n3. Explore each sub-question:\n   - Overfitting occurs when a model becomes too specialized in the training data, leading to poor performance on unseen data. This happens due to the complexity of the model or insufficient data.\n   - Techniques to prevent overfitting include:\n     - Regularization: Adding a penalty term to the loss function encourages simpler models by reducing excessive weights or shrinking coefficients towards zero.\n     - Early stopping: Stopping the training process before it starts to overfit can help maintain good generalization performance. This is a

In [ ]:
def map_to_tot(example):
    q = example["Question"].strip()
    a = example["Answer"].strip()

    has_thinking = "<thinking>" in a and "</thinking>" in a
    has_output   = "<output>" in a and "</output>" in a
    thinking = a.split("<thinking>",1)[1].split("</thinking>",1)[0].strip() if has_thinking else ""
    output   = a.split("<output>",1)[1].split("</output>",1)[0].strip() if has_output else a

    wrapped = (
        "<thinking>\n"
        f"[BRANCH 1]\n{thinking or 'Steps:\n- ...'}\n\n"
        "[SELECTION]\nBest Branch: 1\nWhy: preserves original rationale\nSanity checks: basic\n"
        "</thinking>\n"
        f"<output>\n{output}\n</output>"
    )

    return {
        "prompt": f"Task: {q}\nRespond using <thinking> then <output>.",
        "response": wrapped
    }

# Apply preprocessing
train_ds = raw_ds.map(map_to_tot, remove_columns=raw_ds.column_names)

# Inspect processed sample
print(train_ds[0])


{'prompt': 'Task: "What was the primary technique employed to prevent overfitting during model training?"\nRespond using <thinking> then <output>.', 'response': "<thinking>\n[BRANCH 1]\n1. Break down the question: The main question requires understanding the techniques used to prevent overfitting during model training.\n2. Identify possible sub-questions:\n   - What is overfitting and why does it occur?\n   - What are some common techniques used to prevent overfitting?\n\n3. Explore each sub-question:\n   - Overfitting occurs when a model becomes too specialized in the training data, leading to poor performance on unseen data. This happens due to the complexity of the model or insufficient data.\n   - Techniques to prevent overfitting include:\n     - Regularization: Adding a penalty term to the loss function encourages simpler models by reducing excessive weights or shrinking coefficients towards zero.\n     - Early stopping: Stopping the training process before it starts to overfit c

In [ ]:
SYSTEM_PROMPT = (
    "You are a careful reasoner who solves tasks using a small Tree-of-Thought (ToT) process. "
    "Produce <thinking> with brief branches and a [SELECTION], then give the final <output>. "
    "Prefer correctness and clarity over speed."
)

def map_to_tot(example):
    q = (example.get("Question") or "").strip()
    a = (example.get("Answer") or "").strip()

    has_thinking = "<thinking>" in a and "</thinking>" in a
    has_output   = "<output>"   in a and "</output>"   in a

    thinking = a.split("<thinking>",1)[1].split("</thinking>",1)[0].strip() if has_thinking else ""
    output   = a.split("<output>",1)[1].split("</output>",1)[0].strip()     if has_output   else a

    wrapped = (
        "<thinking>\n"
        "[BRANCH 1]\n"
        f"{thinking or 'Steps:\\n- ...'}\n\n"
        "[SELECTION]\n"
        "Best Branch: 1\n"
        "Why: preserves original rationale\n"
        "Sanity checks: basic\n"
        "</thinking>\n"
        f"<output>\n{output}\n</output>"
    )

    prompt = (
        f"{SYSTEM_PROMPT}\n\n"
        f"Task: {q}\n"
        f"Context:\n"
        f"(If context is empty, proceed with reasonable assumptions.)\n\n"
        "Format your response exactly as:\n"
        "<thinking>...</thinking>\n"
        "<output>...</output>"
    )

    return {"prompt": prompt, "response": wrapped}

# Apply preprocessing
train_ds = raw_ds.map(map_to_tot, remove_columns=raw_ds.column_names)
print(train_ds[0])


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

{'prompt': 'You are a careful reasoner who solves tasks using a small Tree-of-Thought (ToT) process. Produce <thinking> with brief branches and a [SELECTION], then give the final <output>. Prefer correctness and clarity over speed.\n\nTask: "What was the primary technique employed to prevent overfitting during model training?"\nContext:\n(If context is empty, proceed with reasonable assumptions.)\n\nFormat your response exactly as:\n<thinking>...</thinking>\n<output>...</output>', 'response': "<thinking>\n[BRANCH 1]\n1. Break down the question: The main question requires understanding the techniques used to prevent overfitting during model training.\n2. Identify possible sub-questions:\n   - What is overfitting and why does it occur?\n   - What are some common techniques used to prevent overfitting?\n\n3. Explore each sub-question:\n   - Overfitting occurs when a model becomes too specialized in the training data, leading to poor performance on unseen data. This happens due to the comp

In [ ]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer, TrainingArguments
import torch

MODEL_ID = "deepseek-ai/DeepSeek-R1-0528-Qwen3-8B"

# Unsloth loader returns (model, tokenizer)  <-- IMPORTANT
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name         = MODEL_ID,
    max_seq_length     = 2048,
    dtype              = None,        # auto-selects bf16 on Ampere+, else fp16
    load_in_4bit       = True,        # use 4-bit to fit 12–16GB VRAM
    trust_remote_code  = True,
)

# safety: pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

# Attach LoRA (pass hyperparams directly, not a LoraConfig object)
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing="unsloth",  # enables Unsloth smart GC
    random_state=1000,
    use_rslora=False,
    loftq_config=None,
)

print("✅ Model ready with 4-bit + LoRA")


Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.9.1: Fast Qwen3 patching. Transformers: 4.57.0.dev0.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}
Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Model ready with 4-bit + LoRA


In [ ]:
from trl import SFTTrainer

EOS = tokenizer.eos_token or "</s>"
def formatting_func(batch):
    return [
        (p or "").rstrip() + "\n" + (r or "").lstrip() + EOS
        for p, r in zip(batch["prompt"], batch["response"])
    ]

args = TrainingArguments(
    output_dir="./models/r1_tot_unsloth",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    weight_decay=0.01,
    warmup_steps=20,
    max_steps=50,            # smoke test first
    logging_steps=10,
    lr_scheduler_type="linear",
    bf16=(torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8),
    fp16=not (torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8),
    report_to=[],
    save_steps=50,
    seed=1000,
)

trainer = SFTTrainer(
    model=model,                  # <- Unsloth-patched TRL trainer
    train_dataset=train_ds,       # your ['prompt','response'] dataset
    args=args,
    formatting_func=formatting_func,
    max_seq_length=2048,
    dataset_num_proc=1,
    packing=False,
)

trainer.train()


Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/500 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 43,646,976 of 8,234,382,336 (0.53% trained)


Step,Training Loss
10,1.486400
20,1.125300
30,0.796800
40,0.708400
50,0.695900


Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


TrainOutput(global_step=50, training_loss=0.9625777721405029, metrics={'train_runtime': 161.1673, 'train_samples_per_second': 2.482, 'train_steps_per_second': 0.31, 'total_flos': 1.771793514528768e+16, 'train_loss': 0.9625777721405029, 'epoch': 0.8})

In [ ]:
trainer.push_to_hub(commit_message="R1 ToT fine-tune — first push 🚀")
print("✅ Pushed via trainer")

Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...odels/r1_tot_unsloth/tokenizer.json: 100%|#########9| 11.4MB / 11.4MB            

  ...t_unsloth/adapter_model.safetensors:   0%|          |  559kB /  175MB            

  ...ls/r1_tot_unsloth/training_args.bin:   1%|1         |  85.0B / 6.16kB            

✅ Pushed via trainer
